In [1]:
# with hugging face wrapper for sanity check
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from ast import literal_eval
import re

2024-03-05 16:51:56.845308: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 16:51:56.896164: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#training hyperparameters
MAX_TOKENS = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 2
LEARNING_RATE = 1e-05

# change to true to run per review
EXPANDED = False

model_checkpoint = "distilbert/distilbert-base-uncased"

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [1]:
!pwd

/home/sagemaker-user/models


In [4]:
review_df = pd.read_csv('../data/split/train.csv', usecols=['Overall Compliance', 'reviews'])
test_df = pd.read_csv('../data/split/test.csv', usecols=['Overall Compliance', 'reviews'])
val_df = pd.read_csv('../data/split/val.csv', usecols=['Overall Compliance', 'reviews'])
review_df['reviews'] = review_df['reviews'].apply(literal_eval)
test_df['reviews'] = test_df['reviews'].apply(literal_eval)
val_df['reviews'] = val_df['reviews'].apply(literal_eval)

# test classifying at reivew level then resturant level
if EXPANDED:
    review_df = review_df.explode('reviews')
    review_df = review_df.reset_index().drop(columns=['index'])

    test_df = test_df.explode('reviews')
    test_df = test_df.reset_index().drop(columns=['index'])
    
    val_df = val_df.explode('reviews')
    val_df = val_df.reset_index().drop(columns=['index'])

train_ds = Dataset.from_pandas(review_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

review_dataset = DatasetDict()

review_dataset['train'] = train_ds
review_dataset['test'] = test_ds
review_dataset['val'] = val_ds

In [5]:
def clean_reviews(reviews):
    cleaned = []
    for review in reviews:
        review = review.replace('\n', ' ')
        cleaned.append(re.sub(r"[^a-zA-Z0-9]", ' ', review).strip()) #may need to find a better way to do so

    return cleaned

def extract_text_features(data):
    output = {}
    reviews = clean_reviews(data['reviews'])
    output['text'] = " ".join(reviews)
    output['label'] = 0 if data['Overall Compliance'] == 'Yes' else 1

    return output


def extract_expanded_text_features(data):
    output = {}
    review = data['reviews']
    review = review.replace('\n', ' ')
    output['text'] = re.sub(r"[^a-zA-Z0-9]", ' ', review).strip()
    output['label'] = 0 if data['Overall Compliance'] == 'Yes' else 1

    return output

In [6]:
extract_function = extract_text_features

if EXPANDED:
    extract_function = extract_expanded_text_features

review_dataset = review_dataset.map(extract_function)

Map:   0%|          | 0/1754 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize(examples):
    return tokenizer(examples["text"], max_length=MAX_TOKENS, truncation=True)

In [8]:
token_dataset = review_dataset.map(tokenize, remove_columns=['Overall Compliance', 'reviews'])

Map:   0%|          | 0/1754 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)

    return {'accuracy': accuracy, 'recall': recall, 'f1': f1}

In [11]:
id2label = {0: "Pass", 1: "Fail"}
label2id = {"Pass": 0, "Fail": 1}

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
    output_dir="base_bert_model",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_dataset["train"],
    eval_dataset=token_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


train_result = trainer.train()

trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,No log,0.472015,{'accuracy': 0.8205128205128205},{'recall': 0.0},{'f1': 0.0}
2,No log,0.465191,{'accuracy': 0.8205128205128205},{'recall': 0.0},{'f1': 0.0}
3,No log,0.458886,{'accuracy': 0.8205128205128205},{'recall': 0.0},{'f1': 0.0}
4,No log,0.458501,{'accuracy': 0.8205128205128205},{'recall': 0.0},{'f1': 0.0}


***** train metrics *****
  epoch                    =        4.0
  total_flos               =   865563GF
  train_loss               =     0.5201
  train_runtime            = 0:05:07.54
  train_samples_per_second =     22.813
  train_steps_per_second   =      1.431


In [19]:
import json

metrics = trainer.evaluate(token_dataset['val'])

file_path = "base_bert_model/val_results.json"

with open(file_path, "w") as json_file:
    json.dump(metrics, json_file)

In [ ]:
!zip -r base_bert.zip base_bert_model